# Querying and Managing Accounts in VAM Client
This notebook demonstrates how to:
- Perform CRUD (Create, Read, Update, Delete) operations on accounts.
- Filter accounts based on different execution venues 
- Understand the differences in querying `Account`


In [11]:
import os
from pathlib import Path

# Save the original working directory (only once)
try:
    original_wd
except NameError:
    original_wd = os.getcwd()

# Compute the target directory: two levels up from the original working directory
# For example, if original_wd is /Users/username/project/notebooks,
# then target_dir becomes /Users/username
target_dir = Path(original_wd).parents[2]

# Change the working directory to the target directory
os.chdir(target_dir)
print("Working directory set to:", os.getcwd())

import dotenv
dotenv.load_dotenv('.env')

# Import necessary modules
import pandas as pd
import mainsequence.client as msc
from mainsequence.client.utils import MARKETS_CONSTANTS


Working directory set to: /home/jose/code/MainSequenceClientSide/mainsequence-sdk


In [2]:
!pip install matplotlib


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


## Retrieving All Active Accounts
We fetch all active accounts from the database.

### The Main Sequence Execution Venue and Main Sequence Account
The mapping between accounts and execution venues defines which portfolios can be added to specific accounts via the portfolio property `required_venues`. This restriction ensures that an account can only follow portfolios matching the defined `required_venues`. However, as a researcher, you might want to create accounts capable of tracking portfolios across multiple different venues. In such cases, you can utilize accounts within the **Main Sequence ExecutionVenue**, which allows you to mix assets and portfolios from any venue.

In [12]:
# Query all active accounts
accounts= msc.Account.filter()
print(f"Total active accounts: {len(accounts)}")

# Display general account information
for acc in accounts:
    print(f"Account Name: {acc.account_name}, Venue: {acc.execution_venue}, Latest Holdings: {acc.latest_holdings.holdings_date}")

2025-08-07T11:04:05.182275Z [debug    ] took 0.6197 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/account/ application_name=ms-sdk command_id=None data_source_id=2 job_run_id=None project_id=1 (at utils.py:107 in make_request())


Total active accounts: 1
Account Name: Default MainSequence Portfolios Account, Venue: 6, Latest Holdings: 2025-08-04 10:40:41.566892+00:00


# Account Historical Holdings & Risk Factors
We can get historical information as snapshot of our Accounts

In [3]:
import datetime
import matplotlib.pyplot as plt
from mainsequence.client import AccountHistoricalHoldings,AccountRiskFactors


end_date = datetime.datetime.utcnow()
start_date = end_date - datetime.timedelta(hours=1)

# Query with a date range filter
historical_holdings= AccountHistoricalHoldings.filter(
    holdings_date__gte=start_date,
    holdings_date__lte=end_date,
    related_account__id=accounts[0].id
)


# Extract NAV values and dates from historical holdings
nav_values = [holding.nav for holding in historical_holdings if holding.nav is not None]
dates = [holding.holdings_date for holding in historical_holdings if holding.nav is not None]

# Sort data by date
sorted_data = sorted(zip(dates, nav_values), key=lambda x: x[0])

if len(sorted_data) ==0:
    raise Exception("No data found")

dates, nav_values = zip(*sorted_data)

# Plot NAV over time
plt.figure(figsize=(10, 5))
plt.plot(dates, nav_values, marker='o', linestyle='-', label="NAV Over Time")
plt.xlabel("Date")
plt.ylabel("NAV")
plt.title("Account NAV Over Time")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()


2025-08-07T11:02:24.518418Z [debug    ] took 0.3831 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/account_historical_holdings/ application_name=ms-sdk command_id=None data_source_id=2 job_run_id=None project_id=1 (at utils.py:107 in make_request())


Exception: No data found

In [6]:
accounts[0].uuid
target_account=accounts[0]

In [7]:
import mainsequence.client as ms_client

historical_holdings=target_account.get_historical_holdings(start_date=datetime.datetime(2025,6,1))
#pd.DataFrame: A DataFrame indexed by a multi-index of `time_index` (UTC datetime) and `asset_id` (int),
 #      If we want to add extra information relating the assets we need to query the assets as well


assets_in_account=ms_client.Asset.filter(id__in=historical_holdings.index.get_level_values("asset_id").to_list())
id_map={a.id:a.name for a in assets_in_account}
historical_holdings["asset_name"]=historical_holdings.index.get_level_values("asset_id").map(id_map)

2025-08-07T11:03:04.051269Z [debug    ] took 0.9240 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/account_historical_holdings/ application_name=ms-sdk command_id=None data_source_id=2 job_run_id=None project_id=1 (at utils.py:107 in make_request())
2025-08-07T11:03:04.556671Z [debug    ] took 0.5006 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset/ application_name=ms-sdk command_id=None data_source_id=2 job_run_id=None project_id=1 (at utils.py:107 in make_request())


In [8]:
historical_holdings

,,missing_price,price,quantity,asset_name
time_index,asset_id,,,,
2025-08-04 07:26:05.576237+00:00,20668,False,1.0,1000000.0,US Dollar Spot
2025-08-04 07:26:06.727563+00:00,20668,False,1.0,1000000.0,US Dollar Spot
2025-08-04 10:36:50.144099+00:00,20668,False,1.0,1000000.0,US Dollar Spot
2025-08-04 10:39:23.379705+00:00,20668,False,1.0,1000000.0,US Dollar Spot
2025-08-04 10:40:41.566892+00:00,20668,False,1.0,1000000.0,US Dollar Spot
